In [5]:
import pandas as pd

# 1. Load dataset
file_path = "IPL_sample_data_cleaned.csv"
df = pd.read_csv(file_path)

# Convert Runs column to numeric (if it exists)
if "Runs" in df.columns:
    df["Runs"] = pd.to_numeric(df["Runs"], errors="coerce").fillna(0).astype(int)

# 2. Preview data
print("Dataset columns:", df.columns)
print(df.head())

# 3. Select 3 players
players = df['Player Name'].dropna().unique()[:3]
print("Selected Players:", players)

# 4. Define weights
weights = {
    "CP": 1, "GT": 2, "C": 10, "DC": -5,
    "ST": 8, "RO": 12, "MRO": -3, "DH": 6
}

# 5. Function to calculate stats for one player
def player_stats(player, df):
    player_df = df[df['Player Name'] == player]
    
    CP = (player_df['Pick'] == "clean pick").sum()
    GT = (player_df['Pick'] == "good throw").sum()
    C  = (player_df['Pick'] == "catch").sum()
    DC = (player_df['Pick'] == "drop catch").sum()
    ST = (player_df['Throw'] == "stumping").sum()
    RO = (player_df['Throw'] == "run out").sum()
    MRO= (player_df['Throw'] == "missed run out").sum()
    
    if "Short Description" in player_df.columns:
        DH = (player_df['Short Description']
              .str.contains("direct hit", case=False, na=False)).sum()
    else:
        DH = 0
    
    RS = player_df['Runs'].sum() if "Runs" in player_df.columns else 0

    PS = (CP*weights["CP"] + GT*weights["GT"] + C*weights["C"] +
          DC*weights["DC"] + ST*weights["ST"] + RO*weights["RO"] +
          MRO*weights["MRO"] + DH*weights["DH"] + RS)
    
    return {
        "Player": player, "Clean Picks": CP, "Good Throws": GT,
        "Catches": C, "Dropped Catches": DC, "Stumpings": ST,
        "Run Outs": RO, "Missed Run Outs": MRO, "Direct Hits": DH,
        "Runs Saved": RS, "Performance Score": PS
    }

# 6. Apply function to each player
player_stats_list = [player_stats(player, df) for player in players]

# 7. Create DataFrame
performance_df = pd.DataFrame(player_stats_list)

# 8. Display result
print("\nFielding Performance Matrix:\n", performance_df)

# 9. Save to Excel
performance_df.to_excel("Fielding_Performance.xlsx", index=False)
print("\n✅ Saved performance matrix to Fielding_Performance.xlsx")


Dataset columns: Index(['Match No', 'Innings', 'Team', 'Player Name', 'BallCount', 'Position',
       'Pick', 'Dropped Catch', 'Throw', 'Runs', 'Overcount', 'Venue',
       'Stadium'],
      dtype='object')
   Match No  Innings Team     Player Name      BallCount Position  \
0       NaN  IPL2367    1  Delhi Capitals  Rilee russouw      0.1   
1       NaN  IPL2367    1  Delhi Capitals      Phil Salt      0.2   
2       NaN  IPL2367    1  Delhi Capitals     Yash Dhull      0.3   
3       NaN  IPL2367    1  Delhi Capitals     Axer Patel      0.4   
4       NaN  IPL2367    1  Delhi Capitals            NaN      0.5   

               Pick Dropped Catch Throw  Runs Overcount  Venue  \
0  Short mid wicket             n   NaN     1         1  Delhi   
1     wicket keeper             Y     Y     0         1  Delhi   
2            covers             Y     Y     0         1  Delhi   
3             point             Y     Y     0         1  Delhi   
4               NaN           NaN   NaN     0   